In [5]:
import pandas as pd
import numpy as np
from utils import *
pd.set_option('display.max_columns', None)

noaa_austin_path = '../data/Austin_NOAA.csv'
om_austin_path = '../data/Austin_OM.csv'
wrh_austin_path = '../data/Austin_WRH.csv'
air_austin_path = '../data/Austin_Air_Quality.csv'
solar_austin_path = '../data/Austin_Solar_Soil.csv'

noaa_nyc_path = '../data/NYC_NOAA.csv'
om_nyc_path = '../data/NYC_OM.csv'
wrh_nyc_path = '../data/NYC_WRH.csv'
air_nyc_path = '../data/NYC_Air_Quality.csv'
solar_nyc_path = '../data/NYC_Solar_Soil.csv'

noaa_miami_path = '../data/Miami_NOAA.csv'
om_miami_path = '../data/Miami_OM.csv'
wrh_miami_path = '../data/Miami_WRH.csv'
air_miami_path = '../data/Miami_Air_Quality.csv'
solar_miami_path = '../data/Miami_Solar_Soil.csv'

noaa_chicago_path = '../data/Chicago_NOAA.csv'
om_chicago_path = '../data/Chicago_OM.csv'
wrh_chicago_path = '../data/Chicago_WRH.csv'
air_chicago_path = '../data/Chicago_Air_Quality.csv'
solar_chicago_path = '../data/Chicago_Solar_Soil.csv'

daily_df, daily_df_2, daily_df_3, all_df, predictor_final = load_all_dfs(noaa_austin_path, om_austin_path, solar_austin_path, wrh_austin_path, air_austin_path)
#daily_df, daily_df_2, daily_df_3, all_df, predictor_final = load_all_dfs(noaa_nyc_path, om_nyc_path, solar_nyc_path, wrh_nyc_path, air_nyc_path)
#daily_df, daily_df_2, daily_df_3, all_df, predictor_final = load_all_dfs(noaa_miami_path, om_miami_path, solar_miami_path, wrh_miami_path, air_miami_path)
#daily_df, daily_df_2, daily_df_3, all_df, predictor_final = load_all_dfs(noaa_chicago_path, om_chicago_path, solar_chicago_path, wrh_chicago_path, air_chicago_path)

c:\Users\kmfoc\My Drive\BU Files\BU Masters\Work\Spring 2024\CS542\Common Task\organized\models\utils.py:79: DtypeWarning: Columns (13,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [6]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data = all_df

columns_to_ignore = ['date', 'next_day_max_temp']
target_column = 'next_day_max_temp'

X = data.drop(columns=columns_to_ignore)
y = data[target_column]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Add a constant term to the scaled data
X_train_scaled = sm.add_constant(X_train_scaled)
X_test_scaled = sm.add_constant(X_test_scaled)

# Fit the linear regression model on the training data
model = sm.OLS(y_train, X_train_scaled).fit()
print(model.summary())

# Print explained variance on the training data
print("Explained Variance (R-squared) on Training Data:", model.rsquared)

                            OLS Regression Results                            
Dep. Variable:      next_day_max_temp   R-squared:                       0.843
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     1326.
Date:                Mon, 25 Mar 2024   Prob (F-statistic):               0.00
Time:                        05:21:20   Log-Likelihood:                -75347.
No. Observations:               23750   AIC:                         1.509e+05
Df Residuals:                   23653   BIC:                         1.517e+05
Df Model:                          96                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         79.2696      0.038   2110.957      0.0

In [9]:
import plotly.graph_objs as go
from sklearn.metrics import explained_variance_score

# Make predictions on the test data
y_pred = model.predict(X_test_scaled)

print(f'Explained Variance Score: {explained_variance_score(y_test, y_pred):.4f}')

# Print mean squared error on the test data
mse = np.mean((y_pred - y_test) ** 2)
print(f'Mean Squared Error: {mse:.4f}')

# mae
mae = np.mean(np.abs(y_pred - y_test))
print(f'Mean Absolute Error: {mae:.4f}')

# std error
std_error = np.std(y_pred - y_test)
print(f'Standard Error: {std_error:.4f}')

confidence_interval = 1.96 * std_error
print(f'95% Confidence Interval: {confidence_interval:.4f}')


# Create upper and lower bounds for uncertainty shading
upper_bound = y_pred + confidence_interval
lower_bound = y_pred - confidence_interval


# Plot the predicted vs. actual values
# Create traces for actual and predicted values
trace_actual = go.Scatter(
    x=np.arange(len(y_test)),
    y=y_test,
    mode='lines',
    name='Actual'
)

trace_predicted = go.Scatter(
    x=np.arange(len(y_pred)),
    y=y_pred,
    mode='lines',
    name='Predicted'
)

# Create traces for the uncertainty shading
trace_upper_bound = go.Scatter(
    x=np.arange(len(y_pred)),
    y=upper_bound,
    mode='lines',
    marker=dict(color="#444"),
    line=dict(width=0),
    showlegend=False
)

trace_lower_bound = go.Scatter(
    x=np.arange(len(y_pred)),
    y=lower_bound,
    mode='lines',
    marker=dict(color="#444"),
    line=dict(width=0),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty',
    showlegend=False
)

# Create the layout for the figure
layout = go.Layout(
    title='Predicted vs. Actual Values with 95% Confidence Interval',
    xaxis=dict(title='Data Points'),
    yaxis=dict(title='Values')
)

# Create the figure
fig = go.Figure(data=[trace_actual, trace_predicted, trace_upper_bound, trace_lower_bound], layout=layout)

# Display the figure
fig.show()

Explained Variance Score: 0.8313
Mean Squared Error: 38.4151
Mean Absolute Error: 4.5393
Standard Error: 6.1746
95% Confidence Interval: 12.1021
